# Detecting Tree Canopy with Deep Learning When LiDAR Data is Unavailable.

## This script will apply a Deep Learning Model trained on the DeepLabV3 architecture with the Resnet101 backbone. The training data used was Borne Tree Canopy polygons derived from LiDAR. 

## This script will clip a raster to specified ETJ's and classify the pixels with the deep learning model. It will then process the raster into the neccessary components for the UTC application. It will also apply a feature class with 250 points for accuracy assessments. 

## This should only be used where LiDAR is unavailable. Please see the LiDAR Extractor Notebook for further details about LiDAR processing.

In [ ]:
import arcpy
import requests
import os
from arcpy.ia import *
from arcpy.sa import *
# Set environment
arcpy.env.workspace = r"D:\ArcGIS_Projects\UTC\UTC\DeepLearning.gdb"  # Change to your workspace
tiles_layer = "NAIP2022_Index"  # Change to your tiles layer path

# Define base URL for NAIP tiles
base_url = "https://tnris-data-warehouse.s3.us-east-1.amazonaws.com/LCD/collection/naip-2022-nc-cir-60cm/tiles/"
# Output directory for downloaded files
output_base_dir = r"D:/ArcGIS_Projects/UTC/UTC/Temp_Raster_Downloads"  # Change to your desired output directory

# Step 1: Loop through each ETJ polygon
etj_layer = "CollegeStation_ETJ"  # Change to your ETJ layer path
etj_field = "NAME10"  # Field in the ETJ layer that contains the ETJ name

model_path = r"D:\ArcGIS_Projects\UTC\UTC\DeepLearning\Models\DeeplabV3_MassJP2_4Band_ResNet101\DeeplabV3_MassJP2_4Band_ResNet101.dlpk"

In [ ]:
spatial_ref = arcpy.SpatialReference(3665)
with arcpy.da.SearchCursor(etj_layer, [etj_field, "SHAPE@"]) as etj_cursor:
    for etj_row in etj_cursor:
        etj_name = etj_row[0]
        etj_boundary = etj_row[1]
        print(f"Starting Processing for {etj_name}")
        # Create a new folder for this ETJ
        etj_output_dir = os.path.join(output_base_dir, etj_name)
        if not os.path.exists(etj_output_dir):
            os.makedirs(etj_output_dir)

        # Step 2: Select tiles that intersect with the current ETJ polygon
        arcpy.SelectLayerByLocation_management(tiles_layer, "INTERSECT", etj_boundary)
        tile_features = arcpy.CopyFeatures_management(tiles_layer, f'{etj_name}_Tiles')

        # Create a new Mosaic Dataset for this ETJ
        gdb_path = os.path.join(etj_output_dir, f"{etj_name}_raster.gdb")
        if not arcpy.Exists(gdb_path):
            arcpy.management.CreateFileGDB(etj_output_dir, f"{etj_name}_raster.gdb")
        
        #mosaic_dataset = os.path.join(gdb_path, f"{etj_name}_mosaic.tif")
        #arcpy.management.CreateMosaicDataset(gdb_path, f"{etj_name}_mosaic", spatial_ref, num_bands=4, pixel_type="8_BIT_UNSIGNED")

        # Step 3: Create a list of URLs to download
        url_list = []
        raster_files = []
        with arcpy.da.SearchCursor(tile_features, ["NAME", "SHAPE@"]) as cursor:
            for row in cursor:
                tile_field_value = row[0]
                # Assuming file name format is always like 'naip22-nc-cir-60cm_2997544_20220516.jp2'
                if '_' in tile_field_value:
                    parts = tile_field_value.split('_')
                    if len(parts) > 1:
                        tile_number = parts[1][:4]  # Get the first 4 digits
                        tile_url = (base_url + f"{tile_number}/{tile_field_value}")
                        url_list.append(tile_url)
                    else:
                        print(f"Unexpected file name format: {file_name}")
                else:
                    print(f"File name does not contain underscores: {file_name}")

        # Step 4: Download the files
        for url in url_list:
            file_name = url.split('/')[-1]
            local_file_path = os.path.join(etj_output_dir, file_name)
            raster_files.append(local_file_path)
            if os.path.exists(local_file_path):
                print(f"File {local_file_path} already exists, skipping download")
            else:
                try:
                    response = requests.get(url, stream=True)  # Use stream=True for large files
                    response.raise_for_status()  # Check for HTTP errors
                    with open(local_file_path, 'wb') as file:
                        for chunk in response.iter_content(chunk_size=8192):
                            if chunk:
                                file.write(chunk)
                    print(f"Downloaded: {file_name}")
                except requests.exceptions.RequestException as e:
                    print(f"Failed to download {url}: {e}")



        arcpy.management.MosaicToNewRaster(
            input_rasters=raster_files,
            output_location=gdb_path,
            raster_dataset_name_with_extension=f"{etj_name}_mosaic",  # No .tif extension for GDB
            pixel_type="8_BIT_UNSIGNED",
            cellsize=0.60,
            number_of_bands=4
        )



        # Step 6: Clip the mosaic dataset to the ETJ boundary
        clipped_raster_path = os.path.join(etj_output_dir, f"{etj_name}_clipped_raster.tif")
        arcpy.management.Clip(f"{etj_name}_mosaic.tif", "#", clipped_raster_path, etj_boundary, "0", "ClippingGeometry")

        # Step 7: Delete the individual rasters after clipping
        for raster_file in raster_files:
            os.remove(raster_file)
            print(f"Deleted: {raster_file}")

        arcpy.management.SelectLayerByAttribute(tiles_layer, "CLEAR_SELECTION")
        print(f"Processing complete for {etj_name}\n")

print("Imagery Mosaiced for all ETJ polygons.")

In [ ]:
# Get the current project
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_name = "Testing"
# Access the map or layout
map = aprx.listMaps(map_name)[0]  # Adjust index if necessary

# Create an empty list to hold the selected rasters
selected_rasters = []

# Iterate through layers in the map
for layer in map.listLayers():
    # Check if the layer is a raster and ends with '_clipped_raster'
    if layer.isRasterLayer and layer.name.endswith('_clipped_raster.tif'):
        selected_rasters.append(layer)

# Print selected raster names
for raster in selected_rasters:
    print(raster.name)

In [ ]:
for raster in selected_rasters:
    raster_name = raster.name
    base_name = raster_name.split('_')[0]
    print(f"Processing started for {base_name}")
    classified_raster = arcpy.ia.ClassifyPixelsUsingDeepLearning(raster, model_path, "PROCESS_AS_MOSAICKED_IMAGE")
    classified_raster.save(rf"D:\ArcGIS_Projects\UTC\UTC\DLTest\{base_name}_classified.tif")
    
    #Reclassify raster for spatially balanced accuracy assessment points
    raster_calc_result = Con(IsNull(classified_raster), 20, 10)
    raster_calc_result.save(rf"D:\ArcGIS_Projects\UTC\UTC\DLTest\{base_name}_calculated_raster.tif")

    # Run Create Spatial Sampling Locations tool (250 points per strata)
    sampling_output = rf"D:\ArcGIS_Projects\UTC\UTC\DLTest\{base_name}_sampling"
    CreateAccuracyAssessmentPoints(
        in_class_data=raster_calc_result,  # Input classified raster dataset
        out_points=sampling_output,        # Output feature class for accuracy points
        num_random_points=300,      # Number of points
        sampling="EQUALIZED_STRATIFIED_RANDOM"  # Sampling strategy
    )

    # # Convert classified raster to polygons
    polygon_output = rf"D:\ArcGIS_Projects\UTC\UTC\DLTest\{base_name}_Polygons"
    arcpy.conversion.RasterToPolygon(classified_raster, polygon_output, "NO_SIMPLIFY", "VALUE")
    print(f"Processing complete for {base_name}\n")
print("Ready for Accuracy Assessment")